In [8]:
import torch
import torch.nn as nn

from seisLM.data_pipeline import foreshock_aftershock_dataloaders as loaders
from seisLM.model.task_specific import foreshock_aftershock_models as models

dataloaders = loaders.prepare_foreshock_aftershock_dataloaders(
  num_classes=2,
  batch_size=32,
  event_split_method='temporal'
)

Seed set to 42
Seed set to 42


In [10]:
# Example usage
# Input shape: (batch_size, 3, 2500) where 3 is the number of channels and 2500 is the time length
model = models.SeqClassifier(
  in_channels=3,
  num_classes=2,
  num_layers=5,
  initial_filters=32,
  kernel_size=3,
)


In [11]:
from torchinfo import summary

summary(model, input_size=(32, 3, 2500))

Layer (type:depth-idx)                   Output Shape              Param #
SeqClassifier                            [32, 2]                   --
├─Sequential: 1-1                        [32, 512, 73]             --
│    └─DoubleConvBlock: 2-1              [32, 32, 1247]            --
│    │    └─Sequential: 3-1              [32, 32, 1247]            3,488
│    └─DoubleConvBlock: 2-2              [32, 64, 621]             --
│    │    └─Sequential: 3-2              [32, 64, 621]             18,688
│    └─DoubleConvBlock: 2-3              [32, 128, 308]            --
│    │    └─Sequential: 3-3              [32, 128, 308]            74,240
│    └─DoubleConvBlock: 2-4              [32, 256, 151]            --
│    │    └─Sequential: 3-4              [32, 256, 151]            295,936
│    └─DoubleConvBlock: 2-5              [32, 512, 73]             --
│    │    └─Sequential: 3-5              [32, 512, 73]             1,181,696
├─AdaptiveAvgPool1d: 1-2                 [32, 512, 1]         

In [12]:
train_loader = dataloaders['train']

batch = next(iter(train_loader))

In [13]:
out = model(batch[0].cuda())

In [14]:
out


tensor([[0.5096, 0.4904],
        [0.4979, 0.5021],
        [0.4948, 0.5052],
        [0.5014, 0.4986],
        [0.5055, 0.4945],
        [0.4850, 0.5150],
        [0.4887, 0.5113],
        [0.4988, 0.5012],
        [0.5091, 0.4909],
        [0.4763, 0.5237],
        [0.4789, 0.5211],
        [0.4312, 0.5688],
        [0.5077, 0.4923],
        [0.5020, 0.4980],
        [0.4998, 0.5002],
        [0.4959, 0.5041],
        [0.5066, 0.4934],
        [0.5067, 0.4933],
        [0.5066, 0.4934],
        [0.4810, 0.5190],
        [0.5058, 0.4942],
        [0.5080, 0.4920],
        [0.4698, 0.5302],
        [0.0680, 0.9320],
        [0.5030, 0.4970],
        [0.4990, 0.5010],
        [0.5033, 0.4967],
        [0.5069, 0.4931],
        [0.4539, 0.5461],
        [0.4796, 0.5204],
        [0.5051, 0.4949],
        [0.4948, 0.5052]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

In [7]:
batch

[tensor([[[ 3.5783e-03,  2.8276e-03,  3.9459e-03,  ...,  2.5031e-03,
            1.2685e-03,  2.6371e-03],
          [ 1.2586e-03,  2.5097e-03,  7.2960e-05,  ...,  1.9011e-03,
           -9.5287e-04, -2.1381e-03],
          [ 3.8037e-03,  3.8702e-03,  3.7532e-03,  ..., -8.2044e-04,
           -1.3648e-04, -7.3744e-04]],
 
         [[-6.6013e-03,  4.2305e-03,  7.7532e-03,  ...,  3.4833e-03,
            2.5506e-03,  2.4355e-03],
          [-3.2707e-03,  8.8590e-04,  3.0234e-03,  ...,  1.5748e-03,
            1.2092e-03,  7.9352e-04],
          [ 3.4617e-05, -3.3228e-04, -3.1695e-05,  ...,  6.4607e-04,
            1.6141e-03,  1.6144e-03]],
 
         [[ 8.2136e-03, -1.4681e-02, -6.9038e-03,  ...,  3.2647e-02,
            6.5421e-02, -2.5690e-02],
          [ 1.2605e-02,  5.3952e-03,  1.0567e-02,  ..., -3.8728e-02,
            4.0352e-02,  1.4936e-02],
          [-1.5980e-02, -8.4214e-03, -5.6518e-03,  ...,  2.7679e-02,
            1.3561e-02,  5.2242e-02]],
 
         ...,
 
         [[ 